In [ ]:
from utils import * 
from algorithms2 import * 
from agents import *
import warnings 
warnings.filterwarnings('ignore')
numRounds, numPeriods, numSteps = 1, 100, 1
numBuyers, numSellers, numTokens = 1, 1, 5
gameTypes, seed = '1001', 42
disclosure = []
depth = 0
gameData = [gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed]
# ZeroIntelligence, TruthTeller
buyers = [
    Reinforcer(gameData, disclosure, index=0, buyer=1, reinforcer=1, numActions = 2),
]
sellers = [
    TruthTeller(gameData, disclosure, index=0, buyer=0, reinforcer=0),
          ]
log = Log(gameData, disclosure)
verbose = 10
rnd = 0
resetRounds(gameData, log, buyers, sellers, rnd)
print(log.roundData.sellerCosts.item())
print(log.roundData.buyerValues.item())

In [ ]:
#for rnd in range(numRounds):
#    resetRounds(gameData, log, buyers, sellers, rnd)
#    profits = []
for period in range(numPeriods):
    resetPeriods(buyers, sellers)
    profits = []
    for step in range(numSteps):
        resetSteps(buyers, sellers)
        bids, asks = collectOffers(buyers, sellers)
        currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
        price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
        
        bprofit, sprofit = 0, 0
        if price > 0:
            buyers[currentBidIdx].transact(price)
            sellers[currentAskIdx].transact(price)
            bprofit = buyers[currentBidIdx].stepProfits
            sprofit = sellers[currentAskIdx].stepProfits
        profits.append(bprofit)
        log.addStep([rnd, period, step, bids, asks, currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0, bprofit, sprofit])
        observe(buyers, sellers, log.disclose())
        updateStates(buyers, sellers)
    updatePolicy(buyers, sellers)
    #if rnd%verbose==0:
        #print(rnd, period, log.stepData[['currentBidIdx', 'bprofit']].tail(verbose).groupby('currentBidIdx').sum().values.reshape(-1,))
        #print(rnd, period, bids[0], np.sum(profits))

In [1]:
buyers[0].state

NameError: name 'buyers' is not defined

In [ ]:
log.stepData.currentBid.max(), log.stepData.currentBid.min()

In [ ]:
trackBuyersIdx = []
type = 'period'
trackSellersIdx = [0]
rolling_window = 1
df = log.stepData
fig, ax = plt.subplots()
colors = customGraphSettings()
for j in range(numBuyers):
    rolling_mean = df[df.currentBidIdx == j][[type, 'bprofit']].groupby([type]).sum().rolling(rolling_window).mean()
    if j in trackBuyersIdx:
        ax.plot(rolling_mean, color = 'red', alpha = 1.0, linestyle = 'dotted', label = f'Bidder {j}')
    else:
        ax.plot(rolling_mean, color = 'red', alpha = 0.3, linestyle = 'dotted')
        
for j in range(numSellers):
    rolling_mean = df[df.currentAskIdx == j][[type, 'sprofit']].groupby([type]).sum().rolling(rolling_window).mean()
    if j in trackSellersIdx:
        ax.plot(rolling_mean, color = 'blue', alpha = 1.0, linestyle = 'dotted', label = f'Asker {j}')
    else:
        ax.plot(rolling_mean, color = 'blue', alpha = 0.3, linestyle = 'dotted')

ax.set_title('Learning Curves', fontsize=16, fontweight='bold')
plt.xlabel('Period')
plt.ylabel('Avg Profit')
ax.yaxis.tick_left()
ax.xaxis.tick_bottom()
ax.xaxis.set_label_position('bottom')
#ax.set_yticks(np.arange(0, 100, 5))
#ax.set_xticks(np.arange(0, 100, 20))
ax.text(0.80, 0.10, f'Round: {rnd}', transform=plt.gca().transAxes, alpha=0.5)
ax.text(0.80, 0.05, f'Period: {period}', transform=plt.gca().transAxes, alpha=0.5)
ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.15), ncol=5)
plt.ylim(ymin=0)
plt.show()

In [ ]:
rolling_mean.shape[0]

In [ ]:
rolling_mean.shape[0]

In [ ]:
log.graphSales(0, 994, trackBuyersIdx=[], trackSellersIdx=[0])

In [ ]:
log.graphOffers(0, 994, [0], [0])
log.graphSales(0, 994, trackBuyersIdx=[], trackSellersIdx=[0])

In [ ]:
log.graphOffers(0, 0, [0], [0])

In [ ]:
# import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import gym
from collections import deque

# define policy network
class policy_net(nn.Module):
    def __init__(self, nS, nH, nA): # nS: state space size, nH: n. of neurons in hidden layer, nA: size action space
        super(policy_net, self).__init__()
        self.h = nn.Linear(nS, nH)
        self.out = nn.Linear(nH, nA)

    # define forward pass with one hidden layer with ReLU activation and sofmax after output layer
    def forward(self, x):
        x = F.relu(self.h(x))
        x = F.softmax(self.out(x), dim=1)
        return x

# create environment
env = gym.make("CartPole-v1")
# instantiate the policy
policy = policy_net(env.observation_space.shape[0], 20, env.action_space.n)
# create an optimizer
optimizer = torch.optim.Adam(policy.parameters())
# initialize gamma and stats
gamma=0.99
n_episode = 1
returns = deque(maxlen=100)
render_rate = 100 # render every render_rate episodes

while True:
    rewards = []
    actions = []
    states  = []
    # reset environment
    state, _ = env.reset()
    print('state', state)
    while True:
        # render episode every render_rate epsiodes
        if n_episode%render_rate==0:
            env.render()

        # calculate probabilities of taking each action
        probs = policy(torch.tensor(state).unsqueeze(0).float())
        print('probs', probs)

        # sample an action from that set of probs
        sampler = Categorical(probs)
        action = sampler.sample()
        print('action', action)

        # use that action in the environment
        new_state, reward, done, info, _ = env.step(action.item())
        print('new_state', new_state)
        print('reward', reward)
        print('done', done)

        # store state, action and reward
        states.append(state)
        actions.append(action)
        rewards.append(reward)

        state = new_state
        if done:
            break

    # preprocess rewards
    rewards = np.array(rewards)
    # calculate rewards to go for less variance
    batch_rewards = torch.tensor([np.sum(rewards[i:]*(gamma**np.array(range(i, len(rewards))))) for i in range(len(rewards))])
    # or uncomment following line for normal rewards
    print('rewards', rewards)
    print('batch_rewards', batch_rewards)
    print('alt', torch.sum(torch.tensor(rewards)))

    # preprocess states and actions
    batch_states = torch.tensor(states).float()
    batch_actions = torch.tensor(actions)
    print('batch_states', batch_states)
    print('batch_actions', batch_actions)
    
    # calculate gradient
    batch_probs = policy(batch_states)
    print('batch_probs', batch_probs)
    sampler = Categorical(batch_probs)
    batch_log_probs = -sampler.log_prob(batch_actions)   # "-" because it was built to work with gradient descent, but we are using gradient ascent
    print('batch_log_probs', batch_log_probs)

    batch_loss = torch.sum(batch_log_probs * batch_rewards) # loss that when differentiated with autograd gives the gradient of J(θ)
    print('batch_loss', batch_loss.item())
    # update policy weights
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

# close environment
env.close()